In [1]:
import random
import itertools
import glob
import numpy as np
import sparse
from tqdm import tqdm
from PIL import Image
import torch

import matplotlib.pyplot as plt

In [2]:
def create_voxel_grid(cif_file,voxel_count, max_dims, atoms_used=['Ce','O']):
    '''
    Voxelize the a single .cif file. Automatically detects which of 2 .cif file formats is used.
    Inputs:
    cif_file - str, path to .cif file
    voxel_count - list [x,y,z] or scalar if all same, number of voxels per side.
    max_dims - list [x,y,z] of max dimensions across structures. Used to set appropriate
               grid depth in z dimension, which varies across structures.
    atoms_used - list, indicates which atoms to include 
    
    Outputs:
    grid - 3D numpy array [voxel_size_x,voxel_size_y,voxel_size_z] 
           where values indicate number of atoms centered in the voxel
    coor - numpy array [n_atoms,3] of coordinates of each atom
    atom_type numpy array [n_atoms] of atom type
    
    NOTE: coor and atom_type include all atoms, not just those in atoms_used.
    '''
    # Read the .cif file
    f = open(cif_file)
    y = f.readlines()

    # Box dimensions
    # Extract '_cell_length_c' from .cif file. This length varies for each structure.
    increm = 0
    # try-except accounts for new .cif format where cell lengths start on line 3
    try:
        x_dim = float(y[2].split('  ')[-1])
    except ValueError:
        increm = 1
        x_dim = float(y[2+increm].split('  ')[-1])
    y_dim = float(y[3+increm].split('  ')[-1])
    z_dim = float(y[4+increm].split('  ')[-1])

    # Extract the atom locations
    if len(y[15].split("  ")) < 4:

        # for .cif files from ASU_April_21
        if len(y[23].split("  ")) < 4:
            z=np.array([x.split("  ") for x in y[26:]])
            coor = np.array([[float(y) for y in x] for x in z[:,4:7]])
            atom_type = np.array([x.split(" ")[0] for x in z[:,1]])

        # for .cif files from models_wedge_cif and models_wedge_cif_2
        else:
            z=np.array([x.split("  ") for x in y[23:-1]])
            coor = np.array([[float(y) for y in x] for x in z[:,2:5]])
            atom_type = np.array([x.split(" ")[0] for x in z[:,1]])

    # for .cif files from models_cif
    elif len(y[15].split("  ")) == 4:
        z = np.array([x.split("  ") for x in y[15:]]) 
        coor = np.array([[float(y) for y in x] for x in z[:,1:]]) 
        atom_type = z[:,0]

    # Voxelize
    # .cif describes a 1x1x1 box, so voxel_count of 0.25 would create 4 voxels per dimension (4^3 total voxels)
    if type(voxel_count) == list:
        voxel_count_x,voxel_count_y,voxel_count_z = voxel_count
    elif (type(voxel_count) == int) or (type(voxel_count) == float):
        voxel_count_x = voxel_count
        voxel_count_y = voxel_count
        voxel_count_z = voxel_count
        
    # grid has number of channels equal to number of atom types in lattice
    num_atoms = len(atoms_used)
    atom_mapping = {at:i for i,at in enumerate(atoms_used)}
    grid = np.zeros([voxel_count_x,voxel_count_y,voxel_count_z,num_atoms])

    # Normalize coor for axis lengths
    coor = coor*np.array([x_dim,y_dim,z_dim])/np.array(max_dims)
    
    for i,atom in enumerate(coor):
        if atom_type[i] in atoms_used:
            x = int(np.floor(atom[0]*voxel_count_x))
            y = int(np.floor(atom[1]*voxel_count_y))
            z = int(np.floor(atom[2]*voxel_count_z))
            grid[x,y,z,atom_mapping[atom_type[i]]] += 1
    grid = sparse.COO.from_numpy(grid)
    return grid, coor, atom_type

# Determine max z length
def max_z_value(dir_header, dir_list_cif):

    cif_files = []

    for dirr in dir_list_cif:
        cif_files.extend(glob.glob(dir_header+dirr+'/Ce*.cif'))

    x_list,y_list,z_list = [],[],[]

    for cf in cif_files:
        f = open(cf)
        y = f.readlines()
        # increm accounts for new format .cif files
        increm = 0
        try:
            x_list.append(float(y[2].split('  ')[-1]))
        except ValueError:
            increm = 1
            x_list.append(float(y[2+increm].split('  ')[-1]))
        y_list.append(float(y[3+increm].split('  ')[-1]))
        z_list.append(float(y[4+increm].split('  ')[-1]))

    return [np.max(x_list),np.max(y_list),np.max(z_list)]

In [12]:
def generate_training_data(dir_header, dir_list_cif, dir_list_img, voxel_count, 
                           atoms_used, defocus_in, im_size = 256):
    '''
    Generates pairs of voxel grid (X) and image (y) as well as defocus parameter and filenames, for training
    3D grid -> image model.
    
    Inputs:
    dir_list_cif - list of directories containing .cif files, relative path from current directory
    dir_list_img - list of directories containing .yif files, relative path from current directory
    voxel_count - scalar int indicating desired voxels per grid dimension
    atoms_used - List of strings indicating which atom types (e.g. ['O', 'Ce']) to include. 
    defocus_used - List of ints indicating which defocus values to include. If set to 1, all values are used.
    im_size - 2-tuple of ints indicating desired pixels per image dimension (X,Y)
    
    Outputs:
    X_list - List of [grid, atom_type] lists for each .cif-.tif pair used. 
              grid is a [voxel_size^3] np.array with counts of atoms in each voxel
              atom_type is a np.array of strings for each atom's periodic symbol
                
    y_list - List of images, one for each entry in X_list. image is a [im_size[0],im_size[1]] np.array of pixel values.
              image created by cropping input to square then resizing to im_size in PIL.
    defocus_list - List of ints, defocus parameter for each sample 
    img_file_list - List of image filenames for each sample.
    
    '''
    # Calculate largest z value
    max_dims = max_z_value(dir_header,dir_list_cif)
    
    cif_files = []
    for dirr in dir_list_cif:
        cif_files.extend(glob.glob(dir_header+dirr+'/Ce*.cif'))


    X_list = []
    y_list = []
    defocus_list = []
    img_file_list = []

    for k,cif in enumerate(cif_files):
        # Create voxel grid from .cif file 
        X, coor, atom_type = create_voxel_grid(cif,voxel_count, max_dims, atoms_used=atoms_used)

        # Find all associated images
        img_files = []
        if cif.find("CIF") < 0:
            [img_files.extend(glob.glob((dir_header+dirr+cif[cif.find("/",len(dir_header)+3):-4]+'*.tif').replace('[','?').replace(']','?'))) for dirr in dir_list_img]
        else:
            [img_files.extend(glob.glob((dir_header+dirr+cif[cif.find("/",len(dir_header)+len(dirr)+3):-4]+'*.tif').replace('[','?').replace(']','?'))) for dirr in dir_list_img]
        # find starts after the dir_header so it catches the last /
        # Filter out 'def' files
        img_files_clean = []
        for i,im_f in enumerate(img_files):
            if im_f.find("_def_") < 0:
                img_files_clean.append(im_f)
        img_files = img_files_clean        
        # Filter by Defocus value
        nm_loc = [im_f.find("nmDefocus") for im_f in img_files]
        
#         defocus_cif = [abs(int(im_f[loc-2:loc])) for im_f,loc in zip(img_files,nm_loc)]

        defocus_cif = []
        for im_f,loc in zip(img_files,nm_loc):
            if im_f[loc-2:loc-1] == '_':
                defocus_cif.append(abs(int(im_f[loc-1:loc])))
            else:
                defocus_cif.append(abs(int(im_f[loc-2:loc])))
            
    
        if defocus_in == 1:
            defocus_used = set(defocus_cif)
        else:
            defocus_used = defocus_in

        #X_count counts the number of images for each .cif file, to replicate the voxel that many times
        X_count = 0
        # Collect image data
        for defoc,image in zip(defocus_cif,img_files):
            if defoc in defocus_used:
                X_count +=1
                im_data=np.array(Image.open(image).resize((im_size[0],im_size[1])))
                y_list.append(im_data)
                defocus_list.append(defoc)
                img_file_list.append(image)    

        # Make training pairs for each defocus value in defocus_used
        X_list.extend(itertools.repeat(X,X_count)) # Probably move this so it can be repeated for various defocus values
        print("Images to cif {}: {}\t{} ".format(k,X_count,cif))
    return X_list, y_list, defocus_list, img_file_list

In [13]:
# Inputs
# dir_header = '../../em_data/'
# dir_list_cif = ['models_cif','models_wedge_cif','models_wedge_cif_2']
# dir_list_img = ['all_images','all_images_wedge','all_images_wedge_2']

dir_header = 'em_data/'
dir_list_cif = ['models_cif','models_wedge_cif','models_wedge_cif_2',
                '1at/CIF','2at/CIF','2at_2/CIF','3at/CIF',
                '3at_2/CIF','4at/CIF','4at_2/CIF',
                '5at/CIF','5at_2/CIF','6at/CIF','6at_2/CIF','7at/CIF','7at_2/CIF','8at/CIF',
                '8at_2/CIF','9at/CIF','9at_2/CIF','10at/CIF','10at_2/CIF']
dir_list_img = ['all_images','all_images_wedge','all_images_wedge_2',
                '1at','2at','2at_2','3at',
                '3at_2','4at','4at_2','5at','5at_2','6at','6at_2',
                '7at','7at_2','8at','8at_2','9at','9at_2','10at','10at_2']

voxel_count = [84,54,98]
im_size = [84,54]
atoms_used=['Ce','O']
defocus_used = 1 # list defocus values to use, or set =1 to use all

In [10]:
cif = 'jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj/CeO2_[110]_110sur_step_3mu_1-2def_expanded_000x000y000z.cif'
glob.glob((dir_header+'3at_2'+cif[cif.find("/",len(dir_header)+5):-4]+'*.tif').replace('[','?').replace(']','?'))

['em_data/3at_2/CeO2_[110]_110sur_step_3mu_1-2def_expanded_000x000y000z_060slc_2048x2048_2nmDefocus.tif',
 'em_data/3at_2/CeO2_[110]_110sur_step_3mu_1-2def_expanded_000x000y000z_060slc_2048x2048_8nmDefocus.tif',
 'em_data/3at_2/CeO2_[110]_110sur_step_3mu_1-2def_expanded_000x000y000z_060slc_2048x2048_10nmDefocus.tif',
 'em_data/3at_2/CeO2_[110]_110sur_step_3mu_1-2def_expanded_000x000y000z_060slc_2048x2048_6nmDefocus.tif',
 'em_data/3at_2/CeO2_[110]_110sur_step_3mu_1-2def_expanded_000x000y000z_060slc_2048x2048_3nmDefocus.tif',
 'em_data/3at_2/CeO2_[110]_110sur_step_3mu_1-2def_expanded_000x000y000z_060slc_2048x2048_7nmDefocus.tif',
 'em_data/3at_2/CeO2_[110]_110sur_step_3mu_1-2def_expanded_000x000y000z_060slc_2048x2048_9nmDefocus.tif',
 'em_data/3at_2/CeO2_[110]_110sur_step_3mu_1-2def_expanded_000x000y000z_060slc_2048x2048_5nmDefocus.tif',
 'em_data/3at_2/CeO2_[110]_110sur_step_3mu_1-2def_expanded_000x000y000z_060slc_2048x2048_4nmDefocus.tif',
 'em_data/3at_2/CeO2_[110]_110sur_step_3mu_1-

In [ ]:
img_files = []
[img_files.extend(glob.glob((dir_header+dirr+cif[cif.find("/",len(dir_header)+5):-4]+'*.tif').replace('[','?').replace(']','?'))) for dirr in dir_list_img]
# find starts after the dir_header so it catches the last /
# Filter out 'def' files
img_files_clean = []
for i,im_f in enumerate(img_files):
    if im_f.find("_def_") < 0:
        img_files_clean.append(im_f)
img_files = img_files_clean        
# Filter by Defocus value
img_files

In [ ]:
nm_loc = [im_f.find("nmDefocus") for im_f in img_files]

#         defocus_cif = [abs(int(im_f[loc-2:loc])) for im_f,loc in zip(img_files,nm_loc)]
defocus_cif = []
for im_f,loc in zip(img_files,nm_loc):
    if im_f[loc-2:loc-1] == '_':
        defocus_cif.append(abs(int(im_f[loc-1:loc])))
    else:
        defocus_cif.append(abs(int(im_f[loc-2:loc])))

In [ ]:
defocus_used = set(defocus_cif)

#X_count counts the number of images for each .cif file, to replicate the voxel that many times
X_count = 0
# Collect image data
for defoc,image in zip(defocus_cif,img_files):
    if defoc in defocus_used:
        X_count +=1
        im_data=np.array(Image.open(image).resize((im_size[0],im_size[1])))
#         y_list.append(im_data)
#         defocus_list.append(defoc)
#         img_file_list.append(image) 
X_count

In [ ]:
'CeO2_[110]_110sur_step_3mu_1-2def_expanded_000x000y000z'.find("_def_")

In [14]:
X_list, y_list, defocus_list, img_file_list = generate_training_data( \
                            dir_header, dir_list_cif, dir_list_img, voxel_count, atoms_used, defocus_used, im_size)

Images to cif 0: 11	em_data/models_cif/CeO2_011_100sur_8mu.cif 
Images to cif 1: 11	em_data/models_cif/CeO2_011_corner_3mu.cif 
Images to cif 2: 11	em_data/models_cif/CeO2_011_100sur_5mu.cif 
Images to cif 3: 11	em_data/models_cif/CeO2_011_111sur_10mu.cif 
Images to cif 4: 11	em_data/models_cif/CeO2_011_corner_7mu.cif 
Images to cif 5: 11	em_data/models_cif/CeO2_011_cornerb_2mu.cif 
Images to cif 6: 11	em_data/models_cif/CeO2_011_100sur_2mu.cif 
Images to cif 7: 11	em_data/models_cif/CeO2_011_corner_10mu.cif 
Images to cif 8: 11	em_data/models_cif/CeO2_011_111sur_4mu.cif 
Images to cif 9: 11	em_data/models_cif/CeO2_011_111sur_2mu.cif 
Images to cif 10: 11	em_data/models_cif/CeO2_011_100sur_4mu.cif 
Images to cif 11: 11	em_data/models_cif/CeO2_011_cornerb_4mu.cif 
Images to cif 12: 11	em_data/models_cif/CeO2_011_111sur_3mu.cif 
Images to cif 13: 11	em_data/models_cif/CeO2_011_100sur_6mu.cif 
Images to cif 14: 11	em_data/models_cif/CeO2_011_100sur_1mu.cif 
Images to cif 15: 11	em_data/mo

Images to cif 108: 10	em_data/1at/CIF/CeO2_[110]_110sur_step_1mu_0-0def_expanded_-31x295y000z.cif 
Images to cif 109: 10	em_data/1at/CIF/CeO2_[110]_110sur_sawtooh_1mu_0-1def_expanded_000x000y000z.cif 
Images to cif 110: 10	em_data/1at/CIF/CeO2_[110]_110sur_sawtooh_1mu_1-1def_expanded_262x-123y000z.cif 
Images to cif 111: 10	em_data/1at/CIF/CeO2_[110]_110sur_step_1mu_1-1def_expanded_000x000y000z.cif 
Images to cif 112: 10	em_data/1at/CIF/CeO2_[110]_110sur_sawtooh_1mu_0-0def_expanded_218x-393y000z.cif 
Images to cif 113: 10	em_data/1at/CIF/CeO2_[110]_110sur_1mu_1-1def_expanded_-336x231y000z.cif 
Images to cif 114: 10	em_data/1at/CIF/CeO2_[110]_110sur_sawtooh_1mu_0-1def_expanded_284x15y000z.cif 
Images to cif 115: 10	em_data/1at/CIF/CeO2_[110]_110sur_1mu_0-0def_expanded_-241x-044y000z.cif 
Images to cif 116: 10	em_data/1at/CIF/CeO2_[110]_110sur_sawtooh_1mu_0-1def_expanded_284x186y000z.cif 
Images to cif 117: 10	em_data/1at/CIF/CeO2_[110]_110sur_sawtooh_1mu_0-1def_expanded_049x-338y000z.ci

Images to cif 191: 10	em_data/2at/CIF/CeO2_[110]_110sur_2mu_1-2def_expanded_07x232y000z.cif 
Images to cif 192: 0	em_data/2at/CIF/CeO2_[110]_110sur_step_2mu_2-2def_expanded_209x211y000z.cif 
Images to cif 193: 0	em_data/2at/CIF/CeO2_[110]_110sur_step_2mu_2-2def_expanded_129x-201y000z.cif 
Images to cif 194: 10	em_data/2at/CIF/CeO2_[110]_110sur_step_2mu_0-0def_expanded_398x-043y000z.cif 
Images to cif 195: 10	em_data/2at/CIF/CeO2_[110]_110sur_sawtooh_2mu_1-1def_expanded_-072x-179y000z.cif 
Images to cif 196: 10	em_data/2at/CIF/CeO2_[110]_110sur_2mu_1-1def_expanded_363x-109y000z.cif 
Images to cif 197: 10	em_data/2at/CIF/CeO2_[110]_110sur_step_2mu_0-0def_expanded_078x-285y000z.cif 
Images to cif 198: 10	em_data/2at/CIF/CeO2_[110]_110sur_2mu_1-2def_expanded_-169x158y000z.cif 
Images to cif 199: 0	em_data/2at/CIF/CeO2_[110]_110sur_step_2mu_1-2def_expanded_331x228y000z.cif 
Images to cif 200: 10	em_data/2at/CIF/CeO2_[110]_110sur_step_2mu_1-1def_expanded_-395x399y000z.cif 
Images to cif 201:

Images to cif 274: 10	em_data/2at/CIF/CeO2_[110]_110sur_sawtooh_2mu_1-1def_expanded_068x-263y000z.cif 
Images to cif 275: 10	em_data/2at/CIF/CeO2_[110]_110sur_step_2mu_0-0def_expanded_000x000y000z.cif 
Images to cif 276: 10	em_data/2at/CIF/CeO2_[110]_110sur_2mu_0-0def_expanded_-304x-052y000z.cif 
Images to cif 277: 10	em_data/2at/CIF/CeO2_[110]_110sur_sawtooh_2mu_0-0def_expanded_122x-254y000z.cif 
Images to cif 278: 10	em_data/2at/CIF/CeO2_[110]_110sur_step_2mu_0-0def_expanded_291x-043y000z.cif 
Images to cif 279: 10	em_data/2at/CIF/CeO2_[110]_110sur_2mu_2-2def_expanded_163x109y000z.cif 
Images to cif 280: 10	em_data/2at/CIF/CeO2_[110]_110sur_2mu_2-2def_expanded_-033x-38y000z.cif 
Images to cif 281: 0	em_data/2at/CIF/CeO2_[110]_110sur_step_2mu_2-2def_expanded_129x-284y000z.cif 
Images to cif 282: 10	em_data/2at/CIF/CeO2_[110]_110sur_2mu_1-2def_expanded_-169x232y000z.cif 
Images to cif 283: 10	em_data/2at/CIF/CeO2_[110]_110sur_sawtooh_2mu_2-2def_expanded_381x169y000z.cif 
Images to cif 

Images to cif 356: 10	em_data/2at_2/CIF/CeO2_[110]_110sur_sawtooh_2mu_2-2def_expanded_-091x-263y000z.cif 
Images to cif 357: 10	em_data/2at_2/CIF/CeO2_[110]_110sur_2mu_1-1def_expanded_393x357y000z.cif 
Images to cif 358: 10	em_data/2at_2/CIF/CeO2_[110]_110sur_step_2mu_0-0def_expanded_398x226y000z.cif 
Images to cif 359: 10	em_data/2at_2/CIF/CeO2_[110]_110sur_step_2mu_1-1def_expanded_000x000y000z.cif 
Images to cif 360: 10	em_data/2at_2/CIF/CeO2_[110]_110sur_2mu_1-2def_expanded_112x-116y000z.cif 
Images to cif 361: 10	em_data/2at_2/CIF/CeO2_[110]_110sur_sawtooh_2mu_2-2def_expanded_-091x-229y000z.cif 
Images to cif 362: 10	em_data/2at_2/CIF/CeO2_[110]_110sur_2mu_2-2def_expanded_163x-38y000z.cif 
Images to cif 363: 10	em_data/2at_2/CIF/CeO2_[110]_110sur_sawtooh_2mu_1-1def_expanded_068x-179y000z.cif 
Images to cif 364: 10	em_data/2at_2/CIF/CeO2_[110]_110sur_sawtooh_2mu_0-0def_expanded_122x12y000z.cif 
Images to cif 365: 10	em_data/2at_2/CIF/CeO2_[110]_110sur_sawtooh_2mu_1-1def_expanded_000

...

Images to cif 1997: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_sawtooh_9mu_1-1def_expanded_064x-034y000z.cif 
Images to cif 1998: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_step_9mu_2-2def_expanded_-135x054y000z.cif 
Images to cif 1999: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_step_9mu_2-2def_expanded_-135x092y000z.cif 
Images to cif 2000: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_9mu_1-1def_expanded_000x000y000z.cif 
Images to cif 2001: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_step_9mu_1-1def_expanded_-081x-197y000z.cif 
Images to cif 2002: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_step_9mu_0-0def_expanded_094x156y000z.cif 
Images to cif 2003: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_sawtooh_9mu_1-1def_expanded_372x20y000z.cif 
Images to cif 2004: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_9mu_0-0def_expanded_316x-19y000z.cif 
Images to cif 2005: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_9mu_0-0def_expanded_197x-013y000z.cif 
Images to cif 2006: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_9mu_0-0def_expanded_316x-01

Images to cif 2078: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_step_9mu_1-1def_expanded_-214x-197y000z.cif 
Images to cif 2079: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_9mu_1-1def_expanded_-284x-127y000z.cif 
Images to cif 2080: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_sawtooh_9mu_1-2def_expanded_10x198y000z.cif 
Images to cif 2081: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_9mu_2-2def_expanded_-335x099y000z.cif 
Images to cif 2082: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_9mu_1-1def_expanded_062x-356y000z.cif 
Images to cif 2083: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_9mu_0-0def_expanded_316x25y000z.cif 
Images to cif 2084: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_step_9mu_0-0def_expanded_-363x24y000z.cif 
Images to cif 2085: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_step_9mu_2-2def_expanded_113x092y000z.cif 
Images to cif 2086: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_9mu_1-1def_expanded_354x-127y000z.cif 
Images to cif 2087: 10	em_data/9at_2/CIF/CeO2_[110]_110sur_sawtooh_9mu_1-1def_expanded_-042x-034y00

Images to cif 2159: 10	em_data/10at/CIF/CeO2_[110]_110sur_10mu_1-1def_expanded_186x-349y000z.cif 
Images to cif 2160: 10	em_data/10at/CIF/CeO2_[110]_110sur_10mu_1-1def_expanded_000x000y000z.cif 
Images to cif 2161: 10	em_data/10at/CIF/CeO2_[110]_110sur_sawtooh_10mu_2-2def_expanded_182x-133y000z.cif 
Images to cif 2162: 0	em_data/10at/CIF/CeO2_[110]_110sur_step_10mu_1-2def_expanded_054x-046y000z.cif 
Images to cif 2163: 10	em_data/10at/CIF/CeO2_[110]_110sur_sawtooh_10mu_0-0def_expanded_-365x072y000z.cif 
Images to cif 2164: 0	em_data/10at/CIF/CeO2_[110]_110sur_step_10mu_2-2def_expanded_038x279y000z.cif 
Images to cif 2165: 0	em_data/10at/CIF/CeO2_[110]_110sur_step_10mu_2-2def_expanded_000x000y000z.cif 
Images to cif 2166: 10	em_data/10at/CIF/CeO2_[110]_110sur_10mu_1-2def_expanded_197x-104y000z.cif 
Images to cif 2167: 10	em_data/10at/CIF/CeO2_[110]_110sur_10mu_0-0def_expanded_-057x-033y000z.cif 
Images to cif 2168: 10	em_data/10at/CIF/CeO2_[110]_110sur_step_10mu_1-1def_expanded_-178x-15

Images to cif 2239: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_10mu_1-2def_expanded_-352x-387y000z.cif 
Images to cif 2240: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_10mu_1-2def_expanded_000x000y000z.cif 
Images to cif 2241: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_step_10mu_2-2def_expanded_138x279y000z.cif 
Images to cif 2242: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_step_10mu_1-1def_expanded_-243x-159y000z.cif 
Images to cif 2243: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_step_10mu_2-2def_expanded_038x-141y000z.cif 
Images to cif 2244: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_10mu_0-0def_expanded_102x-033y000z.cif 
Images to cif 2245: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_step_10mu_1-2def_expanded_054x393y000z.cif 
Images to cif 2246: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_10mu_1-2def_expanded_231x-387y000z.cif 
Images to cif 2247: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_10mu_2-2def_expanded_-334x242y000z.cif 
Images to cif 2248: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_sawtooh_10mu_0-0def_e

Images to cif 2318: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_step_10mu_1-2def_expanded_261x-182y000z.cif 
Images to cif 2319: 0	em_data/10at_2/CIF/CeO2_[110]_110sur_10mu_0-0def_expanded_-057x-365y000z.cif 
Images to cif 2320: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_10mu_0-0def_expanded_102x-16y000z.cif 
Images to cif 2321: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_sawtooh_10mu_1-1def_expanded_-131x-033y000z.cif 
Images to cif 2322: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_10mu_1-2def_expanded_-352x-104y000z.cif 
Images to cif 2323: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_10mu_1-1def_expanded_-208x041y000z.cif 
Images to cif 2324: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_sawtooh_10mu_1-2def_expanded_-398x-232y000z.cif 
Images to cif 2325: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_step_10mu_1-2def_expanded_-13x-046y000z.cif 
Images to cif 2326: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_step_10mu_0-0def_expanded_-289x117y000z.cif 
Images to cif 2327: 10	em_data/10at_2/CIF/CeO2_[110]_110sur_sawtooh

In [15]:
len(defocus_list)

21117

In [16]:
import pickle

with open("X_list_full.pkl", "wb") as fp:   #Pickling
    pickle.dump(X_list, fp)

with open("y_list_full.pkl", "wb") as fp:   #Pickling
    pickle.dump(y_list, fp)

with open("defocus_list_full.pkl", "wb") as fp:   #Pickling
    pickle.dump(defocus_list, fp)

with open("img_file_list_full.pkl", "wb") as fp:   #Pickling
    pickle.dump(img_file_list, fp)

In [ ]:
def last_filled_slice(grid):
    '''Function returns index of the last non-zero slice in the grid.'''
    # Sum the x-y slices
    slice_sums = grid.sum(axis=0).sum(axis=0)
    # Identify the last non-zero slice
    idx = max(index for index, item in enumerate(slice_sums) if item > 0)
    return idx

In [ ]:
def count_training_data(dir_header, dir_list_cif, dir_list_img,defocus_used):
    '''
    Generates pairs of voxel grid (X) and image (y) as well as defocus parameter and filenames, for training
    3D grid -> image model.
    
    Inputs:
    dir_list_cif - list of directories containing .cif files, relative path from current directory
    dir_list_img - list of directories containing .yif files, relative path from current directory
    voxel_count - scalar int indicating desired voxels per grid dimension
    atoms_used - List of strings indicating which atom types (e.g. ['O', 'Ce']) to include. 
    defocus_used - List of ints indicating which defocus values to include. If set to 1, all values are used.
    im_size - 2-tuple of ints indicating desired pixels per image dimension (X,Y)
    
    Outputs:
    X_list - List of [grid, atom_type] lists for each .cif-.tif pair used. 
              grid is a [voxel_size^3] np.array with counts of atoms in each voxel
              atom_type is a np.array of strings for each atom's periodic symbol
                
    y_list - List of images, one for each entry in X_list. image is a [im_size[0],im_size[1]] np.array of pixel values.
              image created by cropping input to square then resizing to im_size in PIL.
    defocus_list - List of ints, defocus parameter for each sample 
    img_file_list - List of image filenames for each sample.
    
    '''
    cif_files = []
    for dirr in dir_list_cif:
        cif_files.extend(glob.glob(dir_header+dirr+'/Ce*.cif'))

    cif_count = 0
    cif_image_count = []
    image_count = 0
    for cif in cif_files:
        # count the cif files
        cif_count +=1
        # Find all associated images
        img_files = []
        [img_files.extend(glob.glob((dir_header+dirr+cif[cif.find("/",len(dir_header)+5):-4]+'*.tif').replace('[','?').replace(']','?'))) for dirr in dir_list_img]
        # find starts after the dir_header so it catches the last /
        # Filter out 'def' files
        img_files_clean = []
        for i,im_f in enumerate(img_files):
            if im_f.find("_def_") < 0:
                img_files_clean.append(im_f)
        img_files = img_files_clean

        # Filter by Defocus value
        nm_loc = [im_f.find("nmDefocus") for im_f in img_files]
        
#         defocus_cif = [abs(int(im_f[loc-2:loc])) for im_f,loc in zip(img_files,nm_loc)]
        defocus_cif = []
        for im_f,loc in zip(img_files,nm_loc):
            if im_f[loc-2:loc-1] == '_':
                defocus_cif.append(abs(int(im_f[loc-1:loc])))
            else:
                defocus_cif.append(abs(int(im_f[loc-2:loc])))
            
    
        if defocus_used == 1:
            defocus_used = set(defocus_cif)
            
        cif_image_count.append(len(defocus_used))
        image_count += len(defocus_used)
    return cif_count, cif_image_count, image_count

In [ ]:
cif_count, cif_image_count, image_count = count_training_data(dir_header, dir_list_cif, dir_list_img,defocus_used)

In [ ]:
print(".cif count: ",cif_count)
print("image_count: ",image_count)

In [ ]:
defocus_list